In [ ]:
'''
Names (Please write names in <Last Name, First Name> format):
1. Doe, John
2. Doe, Jane

TODO: Semantic segmentation

TODO: Report what each member did in this project

'''

In [20]:
# Mound google drive
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [32]:
import argparse
import torch, torchvision
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import torch.nn as nn
import torch.nn.functional as F

In [33]:
# Commandline arguments
parser = argparse.ArgumentParser()
parser.add_argument('--train_network',
    action='store_true', help='If set, then trains network')
parser.add_argument('--batch_size',
    type=int, default=4, help='Number of samples per batch')
parser.add_argument('--n_epoch',
    type=int, default=1, help='Number of times to iterate through dataset')
parser.add_argument('--learning_rate',
    type=float, default=1e-8, help='Base learning rate (alpha)')
parser.add_argument('--learning_rate_decay',
    type=float, default=0.50, help='Decay rate for learning rate')
parser.add_argument('--learning_rate_decay_period',
    type=float, default=1, help='Period before decaying learning rate')
parser.add_argument('--momentum',
    type=float, default=0.90, help='Momentum discount rate (beta)')
parser.add_argument('--lambda_weight_decay',
    type=float, default=0.0, help='Lambda used for weight decay')

# TODO: please add additional necessary commandline arguments here
parser.add_argument('-f')


args = parser.parse_args()

In [34]:
# Set hyper parameters in notebook
train_network = True #True or False
batch_size = 8
n_epoch = 2
learning_rate = 1e-1
learning_rate_decay = 0.8
learning_rate_decay_period = 10
momentum = 0.5
lambda_weight_decay = 0.5

In [35]:
class FullyConvolutionalNetwork(torch.nn.Module):
    '''
    Fully convolutional network

    Args:
        Please add any necessary arguments
    '''

    def __init__(self):
        super(FullyConvolutionalNetwork, self).__init__()

        # TODO: Design your neural network using
        # (1) convolutional layers
        # https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html
        # (2) max pool layers
        # https://pytorch.org/docs/stable/nn.functional.html#torch.nn.functional.max_pool2d
        # (3) average pool layers
        # https://pytorch.org/docs/stable/nn.functional.html#torch.nn.functional.avg_pool2d
        # (4) transposed convolutional layers
        # https://pytorch.org/docs/stable/generated/torch.nn.ConvTranspose2d.html


        # block 1 - 3x640x480 to 64x320x240
        self.conv1a = nn.Conv2d(3, 64, kernel_size = 3, padding = 1)
        self.conv1b = nn.Conv2d(64, 64, kernel_size = 3, padding = 1)
        self.down1 = nn.MaxPool2d(2,2)

        # block 2 - 64x320x240 to 128x160x120\n",
        self.conv2a = nn.Conv2d(64, 128, kernel_size = 3, padding = 1)
        self.conv2b = nn.Conv2d(128, 128, kernel_size = 3, padding = 1)
        self.down2 = nn.MaxPool2d(2,2)

        # block 3 - 128x160x120 to 256x80x60\n",
        self.conv3a = nn.Conv2d(128, 256, kernel_size = 3, padding = 1)
        self.conv3b = nn.Conv2d(256, 256, kernel_size = 3, padding = 1)
        self.down3 = nn.MaxPool2d(2,2)

        # block 4 - 256x80x60 to 512x40x30\n",
        self.conv4a = nn.Conv2d(256, 512, kernel_size = 3, padding = 1)
        self.conv4b = nn.Conv2d(512, 512, kernel_size = 3, padding = 1)
        self.down4 = nn.MaxPool2d(2,2)

        # block 5 - 512x40x30 to 1024x20x15\n",
        self.conv5a = nn.Conv2d(512, 1024, kernel_size = 3, padding = 1)
        self.conv5b = nn.Conv2d(1024, 1024, kernel_size = 3, padding = 1)
        self.down5 = nn.MaxPool2d(2,2)

        # block 6 - 1024x20x15 to 512x40x30\n",
        self.conv6a = nn.Conv2d(1024, 1024, kernel_size = 3, padding = 1)
        self.conv6b = nn.Conv2d(1024, 1024, kernel_size = 3, padding = 1)
        self.up1 = nn.ConvTranspose2d(1024, 512, kernel_size = 3, padding = 1)

        # block 7 - 512x40x30 to 256x80x60\n",
        self.conv7a = nn.Conv2d(512, 512, kernel_size = 3, padding = 1)
        self.conv7b = nn.Conv2d(512, 512, kernel_size = 3, padding = 1)
        self.up2 = nn.ConvTranspose2d(512, 256, kernel_size = 3, padding = 1)

        # block 8 - 256x80x60 to 128x160x120\n",
        self.conv8a = nn.Conv2d(256, 256, kernel_size = 3, padding = 1)
        self.conv8b = nn.Conv2d(256, 256, kernel_size = 3, padding = 1)
        self.up3 = nn.ConvTranspose2d(256, 128, kernel_size = 3, padding = 1)

        # block 9 - 128x160x120 to 64x320x240\n",
        self.conv9a = nn.Conv2d(128, 128, kernel_size = 3, padding = 1)
        self.conv9b = nn.Conv2d(128, 128, kernel_size = 3, padding = 1)
        self.up4 = nn.ConvTranspose2d(128, 64, kernel_size = 3, padding = 1)

        # block 10 - 64x320x240 to 3x640x480\n",
        self.conv10a = nn.Conv2d(64, 64, kernel_size = 3, padding = 1)
        self.conv10b = nn.Conv2d(64, 64, kernel_size = 3, padding = 1)
        self.up5 = nn.ConvTranspose2d(64, 3, kernel_size = 3, padding = 1)

        self.conv11a = nn.Conv2d(3, 3, kernel_size = 3, padding = 1)
        self.conv11b = nn.Conv2d(3, 3, kernel_size = 3, padding = 1)
        self.conv11c = nn.Conv2d(3, 1, kernel_size = 1, padding = 0)



    def forward(self, x):
        '''
            Args:
                x : torch.Tensor
                    tensor of N x d

            Returns:
                torch.Tensor
                    tensor of n_output
        '''

        # TODO: Implement forward function

        # block 1
        x = self.conv1a(x)
        x = F.relu(x)
        x = self.conv1b(x)
        x = F.relu(x)
        x = self.down1(x)
        
        # block 2
        x = self.conv2a(x)
        x = F.relu(x)
        x = self.conv2b(x)
        x = F.relu(x)
        x = self.down2(x)
        
        # block 3
        x = self.conv3a(x)
        x = F.relu(x)
        x = self.conv3b(x)
        x = F.relu(x)
        x = self.down3(x)
        
        # block 4
        x = self.conv4a(x)
        x = F.relu(x)
        x = self.conv4b(x)
        x = self.down4(x)
        
        # block 5
        x = self.conv5a(x)
        x = F.relu(x)
        x = self.conv5b(x)
        x = F.relu(x)
        x = self.down5(x)
        
        # block 6
        x = self.conv6a(x)
        x = F.relu(x)
        x = self.conv6b(x)
        x = F.relu(x)
        x = self.up1(x)
        
        # block 7
        x = self.conv7a(x)
        x = F.relu(x)
        x = self.conv7b(x)
        x = F.relu(x)
        x = self.up2(x)

        # block 8
        x = self.conv8a(x)
        x = F.relu(x)
        x = self.conv8b(x)
        x = F.relu(x)
        x = self.up3(x)
        
        # block 9
        x = self.conv9a(x)
        x = F.relu(x)
        x = self.conv9b(x)
        x = F.relu(x)
        x = self.up4(x)
        
        # block 10
        x = self.conv10a(x)
        x = F.relu(x)
        x = self.conv10b(x)
        x = F.relu(x)
        x = self.up5(x)
        
        # block 11
        x = self.conv11a(x)
        x = F.relu(x)
        x = self.conv11b(x)
        x = F.relu(x)
        scores = self.conv11c(x)

        return scores

In [42]:
def train(net,
          dataloader,
          n_epoch,
          optimizer,
          learning_rate_decay,
          learning_rate_decay_period):
    '''
    Trains the network using a learning rate scheduler

    Args:
        net : torch.nn.Module
            neural network
        dataloader : torch.utils.data.DataLoader
            # https://pytorch.org/docs/stable/data.html
            dataloader for training data
        n_epoch : int
            number of epochs to train
        optimizer : torch.optim
            https://pytorch.org/docs/stable/optim.html
            optimizer to use for updating weights
        learning_rate_decay : float
            rate of learning rate decay
        learning_rate_decay_period : int
            period to reduce learning rate based on decay e.g. every 2 epoch

        Please add any necessary arguments

    Returns:
        torch.nn.Module : trained network
    '''

    # TODO: Define loss function
    loss_func = torch.nn.CrossEntropyLoss()

    for epoch in range(n_epoch):

        # Accumulate total loss for each epoch
        total_loss = 0.0

        # TODO: Decrease learning rate when learning rate decay period is met
        # e.g. decrease learning rate by a factor of decay rate every 2 epoch
        if epoch and epoch % learning_rate_decay_period == 0:

          for param_group in optimizer.param_groups:
            param_group['lr'] = learning_rate_decay * paramgroup['lr']

        for batch, (images, segmentations) in enumerate(dataloader):

          # TODO: Forward through the network
          outputs = net(images)

          # TODO: Clear gradients so we don't accumlate them from previous batches
          optimizer.zero_grad()

          # TODO: Compute loss function
          loss = loss_func(outputs, segmentations)

          # TODO: Update parameters by backpropagation
          loss.backward()
          optimizer.step()

          # TODO: Accumulate total loss for the epoch
          total_loss = total_loss + loss.item()

        mean_loss = total_loss / float(batch)

        # Log average loss over the epoch
        print('Epoch=%d  Loss: %.3f' % (epoch + 1, mean_loss))

    return net

In [37]:
def evaluate(net, dataloader):
    '''
    Evaluates the network on a dataset

    Args:
        net : torch.nn.Module
            neural network
        dataloader : torch.utils.data.DataLoader
            # https://pytorch.org/docs/stable/data.html
            dataloader for training data

        Please add any necessary arguments
    '''
    IOU = []

    # Make sure we do not backpropagate
    with torch.no_grad():

        for (images, segmentations) in dataloader:

            # TODO: Forward through the network

            outputs = net(images)

            # Take the argmax over the outputs
            _, predictions = torch.max(outputs, dim=1)

            # TODO: Compute evaluation metric(s) for each sample (IOU)
            #outputs = net(images)
            iou = intersection_over_union(predictions, segmentations)
            IOU.append(iou)

    # TODO: Compute mean evaluation metric(s)
    mean_iou = sum(IOU) / len(IOU)

    # TODO: Print scores
    print('Mean accuracy: %d' % mean_iou)

    # TODO: Convert the last batch of images back to original shape
    images = images.view(shape[0], shape[1], shape[2], shape[3])
    images = images.cpu().numpy()
    images = np.transpose(images, (0, 2, 3, 1))

    # TODO: Convert the last batch of predictions to the original image shape

    # TODO: Plot images
    plot_images(images, 5, 5, '...')

    plt.show()

In [38]:
def intersection_over_union(prediction, ground_truth):
    '''
    Computes the intersection over union (IOU) between prediction and ground truth

    Args:
        prediction : numpy
            N x h x w prediction
        ground_truth : numpy
            N x h x w ground truth

    Returns:
        float : intersection over union
    '''

    # TODO: Computes intersection over union score
    # Implement ONLY if you are working on semantic segmentation

    # Flatten the prediction and ground_truth
    p = torch.flatten(prediction)
    gt = torch.flatten(ground_truth)

    # call sklearn.metrics.jaccard_score to get IoU
    return sklearn.metrics.jaccard_score(p, gt)

In [ ]:
def mean_squared_error(prediction, ground_truth):
    '''
    Computes the mean squared error (MSE) between prediction and ground truth

    Args:
        prediction : numpy
            N x h x w prediction
        ground_truth : numpy
            N x h x w ground truth

    Returns:
        float : mean squared error
    '''

    # TODO: Computes mean squared error
    # Implement ONLY if you are working on image reconstruction or denoising

    return 0.0

In [39]:
def plot_images(X, Y, n_row, n_col, fig_title):
    '''
    Creates n_row by n_col panel of images

    Args:
        X : numpy
            N x h x w input data
        Y : numpy
            N x h x w predictions
        n_row : int
            number of rows in figure
        n_col : list[str]
            number of columns in figure
        fig_title : str
            title of plot

        Please add any necessary arguments
    '''

    fig = plt.figure()
    fig.suptitle(fig_title)

    # TODO: Visualize your input images and predictions
    for i in range(1, n_row * n_col + 1):
      ax = fig.add_subplot(n_row, n_col, i)
      index = i - 1
      x_i = X[index, ...]
      subplot_title_i = subplot_title[index]
      if len(x_i.shape) == 1:
        x_i = np.expand_dims(x_i, axis=0)
      ax.set_title(subplot_title_i)
      ax.imshow(x_i)

      plt.box(False)
      plt.axis('off')


In [29]:
# TODO: Set up data preprocessing step
# https://pytorch.org/docs/stable/torchvision/transforms.html
data_preprocess_transform = torchvision.transforms.Compose([
    torchvision.transforms.Resize(size=(256, 256)),
    torchvision.transforms.ToTensor()
])

# Download and setup your training set
dataset_train = torchvision.datasets.VOCSegmentation(
    root='/content/gdrive/MyDrive/ELEC535final',
    image_set='train',
    download=True,
    transform=data_preprocess_transform,
    target_transform = data_preprocess_transform
  )

# Setup a dataloader (iterator) to fetch from the training set
dataloader_train = torch.utils.data.DataLoader(
    dataset_train,
    batch_size=8,
    shuffle=True,
    num_workers=0
)

# Download and setup your validation/testing set
dataset_val = torchvision.datasets.VOCSegmentation(
    root='/content/gdrive/MyDrive/ELEC535final',
    image_set='val',
    download=True,
    transform=data_preprocess_transform,
    target_transform = data_preprocess_transform
)

# TODO: Setup a dataloader (iterator) to fetch from the validation/testing set
dataloader_val = torch.utils.data.DataLoader(
    dataset_val,
    batch_size=8,
    shuffle=True,
    num_workers=0
)

Using downloaded and verified file: /content/gdrive/MyDrive/ELEC535final/VOCtrainval_11-May-2012.tar
Using downloaded and verified file: /content/gdrive/MyDrive/ELEC535final/VOCtrainval_11-May-2012.tar


In [40]:
# TODO: Define network
net = FullyConvolutionalNetwork()

# TODO: Setup learning rate optimizer
# https://pytorch.org/docs/stable/optim.html?#torch.optim.SGD
optimizer = torch.optim.SGD(
    net.parameters(),
    lr = learning_rate,
    weight_decay = lambda_weight_decay,
    momentum=momentum
)

In [43]:
if train_network:

  # Set network to training mode
  net.train()

  # TODO: Train network and save into checkpoint
  net = train(
      net = net,
      dataloader = dataloader_train,
      optimizer = optimizer,
      learning_rate_decay = learning_rate_decay,
      learning_rate_decay_period = learning_rate_decay_period,
      n_epoch = n_epoch
  )

  torch.save({'state_dict' : net.state_dict()}, '/content/drive/My Drive/ELEC535final/checkpoint.pth')

else:
  # Load network from checkpoint
  checkpoint = torch.load('/content/drive/My Drive/ELEC535final/checkpoint.pth')
  net.load_state_dict(checkpoint['state_dict'])

RuntimeError: ignored

In [ ]:
# Set network to evaluation mode
net.eval()

# TODO: Evaluate network on testing set
evaluate(
    net = net,
    dataloader = dataloader_test
)

In [22]:
'''if __name__ == '__main__':

    # TODO: Set up data preprocessing step
    # https://pytorch.org/docs/stable/torchvision/transforms.html
    data_preprocess_transform = torchvision.transforms.Compose([
        torchvision.transforms.Resize(size=(256, 256)),
        torchvision.transforms.ToTensor()
    ])

    # Download and setup your training set
    dataset_train = torchvision.datasets.VOCSegmentation(
        root='/content/gdrive/MyDrive/ELEC535final',
        image_set='train',
        download=True,
        transform=data_preprocess_transform,
        target_transform = data_preprocess_transform
    )

    # Setup a dataloader (iterator) to fetch from the training set
    dataloader_train = torch.utils.data.DataLoader(
        dataset_train,
        batch_size=8,
        shuffle=True,
        num_workers=0
    )

    # Download and setup your validation/testing set
    dataset_val = torchvision.datasets.VOCSegmentation(
        root='./data',
        image_set='val',
        download=True,
        transform=data_preprocess_transform,
        target_transform = data_preprocess_transform
    )

    # TODO: Setup a dataloader (iterator) to fetch from the validation/testing set
    dataloader_val = torch.utils.data.DataLoader(
        dataset_val,
        batch_size=8,
        shuffle=True,
        num_workers=0
    )



    # TODO: Define network
    net = FullyConvolutionalNetwork()

    # TODO: Setup learning rate optimizer
    # https://pytorch.org/docs/stable/optim.html?#torch.optim.SGD
    optimizer = torch.optim.SGD(
        net.parameters(),
        lr = learning_rate,
        weight_decay = lambda_weight_decay,
        momentum=momentum
    )

    if train_network:
        # Set network to training mode
        net.train()

        # TODO: Train network and save into checkpoint
        net = train(
            net = net,
            dataloader = dataloader_train,
            optimizer = optimizer,
            learning_rate_decay = learning_rate_decay,
            learning_rate_decay_period = learning_rate_decay_period
        )

        torch.save({'state_dict' : net.state_dict()}, '/content/drive/My Drive/ELEC535final/checkpoint.pth')

    else:
        # Load network from checkpoint
        checkpoint = torch.load('/content/drive/My Drive/ELEC535final/checkpoint.pth')
        net.load_state_dict(checkpoint['state_dict'])

    # Set network to evaluation mode
    net.eval()

    # TODO: Evaluate network on testing set
    evaluate(
        net = net,
        dataloader = dataloader_test
    )'''

Using downloaded and verified file: ./data/VOCtrainval_11-May-2012.tar


ModuleAttributeError: ignored